# Загрузка Pandas и очистка данных

In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('main_task.csv')

df_test = pd.read_csv('kaggle_task.csv')
sample_submission = pd.read_csv('sample_submission.csv')

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
df.head()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               0 non-null      float64
 2   Cuisine Style      0 non-null      float64
 3   Ranking            0 non-null      float64
 4   Rating             0 non-null      float64
 5   Price Range        0 non-null      float64
 6   Number of Reviews  0 non-null      float64
 7   Reviews            0 non-null      float64
 8   URL_TA             0 non-null      float64
 9   ID_TA              0 non-null      float64
dtypes: float64(9), object(1)
memory usage: 3.1+ MB


In [10]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

#Сохраним данные о наличии пропусков в отдельные признаки и заполним их
df['cuisine_style_isNAN'] = pd.isna(df['Cuisine Style']).astype('uint8')
df['number_of_reviews_isNAN'] = pd.isna(df['Number of Reviews']).astype('uint8')

df['Cuisine Style']=df['Cuisine Style'].fillna("")
df = df.fillna(0)


In [11]:
#URL_TA','ID_TA', 'Reviews -удалим из-за малой информативности их
df=df.drop(columns=['URL_TA','ID_TA', 'Reviews'])
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,cuisine_style_isNAN,number_of_reviews_isNAN
0,"id_5569,Paris,""['European', 'French', 'Interna...",0.0,,0.0,0.0,0.0,0.0,1,1
1,"id_1535,Stockholm,,1537.0,4.0,,10.0,""[['Unique...",0.0,,0.0,0.0,0.0,0.0,1,1
2,"id_352,London,""['Japanese', 'Sushi', 'Asian', ...",0.0,,0.0,0.0,0.0,0.0,1,1
3,"id_3456,Berlin,,3458.0,5.0,,3.0,""[[], []]"",/Re...",0.0,,0.0,0.0,0.0,0.0,1,1
4,"id_615,Munich,""['German', 'Central European', ...",0.0,,0.0,0.0,0.0,0.0,1,1


In [12]:
#Заменим средним значением рейтинга по городу
def calc_mean_score(row):
    return df[df['City']==row['City']]['Rating'].mean()

In [13]:
df['city_mean']=df.apply(calc_mean_score, axis=1)

In [14]:
#Разобьем Price Range на столбцы low_price для $, mid_price для $$-$$$, high_price для $$$$
df['Price Range'].value_counts()
df.head()

df=pd.get_dummies(df, columns=['Price Range'])
df.columns=['Restaurant_id','city', 'cuisine_style','ranking','Rating', 'review_numbers', 'cuisine_style_isNAN', 'number_of_reviews_isNAN', 'city_mean','low_price','mid_price','high_price','no_price']
df

ValueError: Length mismatch: Expected axis has 10 elements, new values have 13 elements

In [ ]:
#cuisine_style - столбец со списком предлагаемых в ресторане видов кухни. Посчитаем сколько видов кухонь предлагается в каждом из ресторанов
df.cuisine_style.value_counts().head(30)

In [ ]:
pd.Series(cuisine).value_counts().sort_values()
count = []
for i in df.cuisine_style.values:
    if i == 0:
        count.append(1)
    else:
        count.append(len(i.split(',')))

df['count_cuisine'] = count
df.head()

In [ ]:
#ranking - обозначает место, которое ресторан занимает среди ресторанов города. Определяем поправку в каком городе ресторан может быть хуже среднего ресторана в другом городе
df['rest_counts'] = df.city.apply(lambda x: df.city.value_counts()[x])
df['relative_rank'] = df.ranking / df.rest_counts

In [ ]:
#Уберем столбец ranking так как мы его заменили на relative_rank, а также вспомогательный столбец rest_counts. И оставшиеся нечисловые стобцы, которые мы также заменили 'city','restaurant_id','cuisine_style'

df=df.drop(columns=['ranking','rest_counts','city','cuisine_style'])
df

In [ ]:
#Cильнокоррелирующих столбцов нет, поэтому можно оставить все
sns.heatmap(df.corr())

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
test_data = df.drop(['Rating','Restaurant_id'], axis=1)

In [ ]:
predict_submission = regr.predict(test_data[:10000])
predict_submission = np.round(predict_submission*2)/2
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head(10)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))